### Generator wykresów 

#### Generator opiera się na 4 pętlach for. Pętle bazują kolejno na: klasyfikatorach, cechach, metodach liczenia wyniku, statystykach. Do stworzenia jednego wykresu potrzeba wszystkich statystyk - czyli pełnego obiegu pętli 4-tej. W funkcji initDF trzeba wpisać ścieżkę z której pobierane są zdjęcia ( aktualnie jest to ustawione tak jak Faustyna dodała wektory 1d (każdy rodzaj jest w innym folderze)). Hierarchia:

###### 1dNew
######  ---------5th_percentile 
###### ------------------------------ 1d vectors
###### ---------median 
######  -------------------------------1d vectors 
###### etc.

#### Do przemyślenia są zmienne globalne które zmienie, ale narazie nie mam pomysłu jak to inaczej rozwiązać. Wykresy tworzą się bezpośrednio w folderze w którym jest wklejony plik jupytera.

#### Klasyfikatory mają wpisane opytmalne wartości policzone dzięki  Grid & Random Serach (choć nie wsyzstkie nad czym pracuje) - być może połączę liczenie optymalnych wartości dla klasyfikatorów wraz z poniższą częścią, choć nie wiem czy jest sens - po ustaleniu, że np. RandomForest najlepiej działa dla no_estimatorsimators = 70, to średnio jest sens znowu to udowadniać. 

#### Mój angielski jest średni ale specjalnie pisałem komentarze niepolskie bo cza kiedyś zacząć. Jak coś jest po chińsku, a nie po ang to po konsultacji autor może powiedzieć co miał na myśli. 


In [0]:
import pandas as pd
import cv2 as cv
import numpy as np
import glob 
import sys
#import statistics
import matplotlib.pyplot as plt
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score 

In [0]:
#@title Classifier Parameters

result_plot_dir = "/home/reflex/reflex/result_plots/" #@param {type:"string"}
joint_csv_path = '/home/reflex/reflex/joint_file.csv' #@param {type:"string"}
no_estimators = 70 #@param {type:"raw"}
no_cv_folds = 10 #@param {type:"raw"}
no_jobs = 8 #@param {type:"raw"}
no_neighbours = 3 #@param {type:"raw"}


## Global variables and constatnts

In [0]:
statistics = ["min" , "max", "var", "median", "mean","95th_percentile","5th_percentile"]
scoring_types = ['recall_macro','accuracy','precision','recall','f1','roc_auc']
class_names = ["Loop scattering","Background ring","Strong background","Diffuse scattering","Artifact","Ice ring","Non-uniform detector"]
classifier_names = ["RFC","DTC","KNN","GaussianNB","QuadraticDisciminantAnalysis", "GPC", "SVC"]

In [0]:
def get_classifier_filename(classifier_name, class_name):
  
    if classifier_name not in classifier_names:
      raise Exception("Invalid classifier name!")
  
    return dict(zip(classifier_names, [
      result_plot_dir + "RFClf-cv"+str(no_cv_folds)+"-n_est"+str(no_estimators)+"-"+class_name+".jpg",
      result_plot_dir + "DTClf-cv"+str(no_cv_folds)+class_name+".jpg",
      result_plot_dir + "KNNClf-cv"+str(no_cv_folds)+"-neighbours-"+str(no_neighbours)+"-"+class_name+".jpg",
      result_plot_dir + "GaussClf-cv"+str(no_cv_folds)+"-"+class_name+".jpg",
      result_plot_dir + "QDAClf-cv"+str(no_cv_folds)+"-"+class_name+".jpg",
      result_plot_dir + "GaussPrc-cv"+str(no_cv_folds)+"-"+class_name+".jpg",
      result_plot_dir + "SV-cv"+str(no_cv_folds)+"-"+class_name+".jpg" 
    ]))[classifier_name]
    

'''
Constructs an appropriate classifier object for the given classifier name.
'''
def get_classifier(classifier_name):
  
    if classifier_name not in classifier_names:
      raise Exception("Invalid classifier name!")
    
    classifier_objects = [
        RandomForestClassifier(random_state=23, n_estimators=no_estimators, n_jobs=no_jobs),
        DecisionTreeClassifier(random_state=10),
        KNeighborsClassifier(n_neighbors=no_neighbors, n_jobs=no_jobs),
        GaussianNB(),
        QuadraticDiscriminantAnalysis(),
        GaussianProcessClassifier(1.0 * RBF(1.0)),
        SVC(gamma=2, C=1, probability="True")
    ]
        
    return dict(zip(classifier_names, classifier_objects))[classifier_name]

In [0]:
def create_joint_vector(image, directory, statistics=None, all_statistics=False, sort=False, 
                        sort_function=lambda x: x, vector_length=None, 
                        fill_value=None, image_as_index=True, predefined_file_list=None, 
                        remove_suffix=False):

  """
  Tworzy dataframe z wybranego zdjecia w postaci <nazwa_zdjecia> <wektor_zlaczonych_statystyk>
  
  Params:
  --------------
    image                 - nazwa pliku ze zdjeciem (z suffixem (.SSSxSSS.png))
    directory             - katalog z katalogami ze wszystkimi statystykami
    statistics            - lista statystyk branych pod uwage
    all_statistics        - jezeli True to poprzedni parametr jest ignorowany i pod uwage bierzemy
                            wszystkie statystyki we wskazanym folderze
    sort                  - jezeli True to nazwy statystyk sortowane sa wedlug podanej funkcji
                            sortujacej (domyslnie leksykograficznie)
    sort_function         - funkcja (key) sortujaca statystyki (domyslnie leksykograficznie)
    vector_length         - parametr okreslajacy pozadana dlugosc wektora. W przypadku 
                            nadmiaru jest przycinany, w przeciwnym razie jest wypelniniany
                            kolejnym paremetrem. Niezdefiniowany (None) nie modyfikuje wektora.
    fill_value            - wartosc, ktora wypelniany bedzie wektor, jezeli będzie za krótki
                            w przypadku zdefiniowania dlugości
    image_as_index        - ustawie nazwe pliku jako index DataFrame'u
    remove_suffix         - usuwa suffix (.SSSxSSS.png) z nazwy pliku
    
                     
   Returns:
   --------------
    None                  - w przpadku bledu (brak zdef. statystyk, zly katalog, zla nazwa pliku)
    Dataframe             - kiedy wszystko poszlo zgodnie z zalozeniami
                    
                      
  """
  
  # Normalizacja sciezki do foldery
  if directory[-1] != '/':
    directory += '/'
    
  # Sprawdzamy czy mamy jakiekolwiek statystyki do złączenia
  statistic_names = list(os.listdir(directory)) if all_statistics else statistics 

  if statistic_names:   

    # Aby uniknąć niedeterminizumu (?) w przypadku listdir sortujemy sortujemy statystyki leksykograficznie
    if sort:
      statistic_names.sort(key=sort_function)

    values = []

    # Laczenie wektora
    for stat_name in statistic_names:

      current_vector = cv.imread(f'{directory}{stat_name}/{image}', cv.IMREAD_GRAYSCALE).flatten().tolist()
      
      # Dostosowywanie dlugosci wektora
      if vector_length:
        current_vector = current_vector[:vector_length]
        current_vector.extend([fill_value] * max(0, vector_length - len(current_vector)))
      
      values.append(current_vector)

    # Tworzenie dataframe
    
    if remove_suffix:
      image = image[:-12]
      
    data = []
    column_names = ['img']
    for idx, stat_vec in enumerate(values):
      column_names += [f'{statistic_names[idx]}_{i}' for i in range(len(stat_vec))]
      data += stat_vec
      
    df = pd.DataFrame([[image, *data]], columns=column_names)

    # Ustawianie nowego indexu
    if image_as_index:
      df.set_index('img', inplace=True)

    return df
      
  return None


'''
Constructs a DataFrame from all vector files in the given directory.
'''
def vector_folder_to_df(directory, limit=None):
  
  files = list(os.listdir(directory + '/' + list(os.listdir(directory))[0]))
  all_dfs = []

  for idx, image in enumerate(files[:limit]): 
    all_dfs.append(create_joint_vector(image, directory, all_statistics=True, sort=True, vector_length=240, remove_suffix=True, image_as_index=True))
    
  return pd.concat(all_dfs)


def extract_filename(path):
    path, filename = os.path.split(path)
    return '.'.join(filename.split('.')[:-1])

  
def construct_joint_csv(output_filepath):
  
  df1 = pd.read_csv('/home/reflex/reflex/data/results_constant_vector_length/vectors/vectors.csv')
  df1.set_index('img', inplace=True, drop=True)
  
  df2 = pd.read_csv('/home/reflex/reflex/reflex.csv')
  df2['Image'] = df2['Image'].apply(lambda x: extract_filename(x))
  df2.set_index('Image', inplace=True, drop=True)

  joint = pd.concat([df1, df2], axis=1, join='inner')
  joint.to_csv(output_filepath)
  return joint

In [0]:
def classify(X, y, classifier_name, scoring):    
  
    # TODO now clf automaticly divide into train/test set, but most of clfs takes divided sets
    #X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.1, random_state=42)
    #print(f'Training set size: {len(X_train)} & Test set size: {len(X_test)}')
    ### cv - cross-validation generator - default KFold(n_splits, shuffle, random state) splits into K folds 
    ### it trains on K-1, test on 'untouched' 1 part
    
    clf = get_classifier(classifier_name)
    return cross_val_score(clf, X, y, cv=no_cv_folds, scoring=scoring)

In [0]:
def plot(data, classifier_name, class_name, scoring_names):    
    
    f, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(12,12))
    ax1.set_title(f'{class_name}-{classifier_name}-{no_cv_folds}folds')
    
    x = []
    for i in range(len(data)):
      for j in range(len(data[0])):
        x.append(i+1)
    y = np.array(data).flatten()

    color = np.tile(np.arange(len(data[0])), len(scoring_names))
    scatter = ax1.scatter(x, y, s=20, c=color)
    cbar = plt.colorbar(scatter, ax=[ax1, ax2])
    cbar.set_ticks(np.arange(len(data[0])))
    
    ax2.boxplot(data, 0, 'gD', 1)
    
    ax2.set_xticklabels(scoring_names, rotation=45)
        
    plt.show()
    f.savefig(get_classifier_filename(classifier_name, class_name))

In [0]:
def initDF(class_name):
        
    joined = pd.read_csv(joint_csv_path)
    print("Dataset size: ", len(joined))
    
    y = joined.loc[:, class_name].values
    X = joined.as_matrix(columns=joined.columns[1:-7])
    
    return X, y  

In [0]:
def initDF_augmented(class_name):
        
    joined = pd.read_csv(joint_csv_path)
    print("Dataset size: ", len(joined))
    
    
    for index, row in joined.iterrows():
      print(index)
      print(row)
    
    y = joined.loc[:, class_name].values
    X = joined.as_matrix(columns=joined.columns[1:-7])
    
    return X, y  

In [0]:
def main():
      
    for classifier_name in classifier_names: # iterate over classifiers (RFC,DTC ...)
        print("Classifier:", classifier_name)  
        
        for class_name in class_names:  # iterate over classes (Strong Bg, Loop Scattering, ... ) 
            print(class_name, end=' ')
            #X, y = initDF(class_name)
            X, y = initDF_augmented(class_name)
            
            scores = []
            for scoring in scoring_types[:2]:  # iterate over scoring types (Accuracy, Recall ...)
                print('.', end='')
                result = classify(X, y, classifier_name, scoring) #FIXME runs classifier to calculate each score
                scores.append(result)
            
            plot(scores, classifier_name, class_name, scoring_types) # plot all stats on specific: CLF , FEATURE , SCORING_TYPE 

In [342]:
#construct_joint_csv(joint_csv_path)
main()

Classifier: RFC
Loop scattering 

KeyboardInterrupt: ignored